# Step-Back Prompting (Question-Answering)

You can also run this notebook online [on Noteable.io](https://app.noteable.io/published/ff04b7ba-efaf-48d5-ba4d-b6bff0359ceb).

One prompting technique called "Step-Back" prompting can improve performance on complex questions by first asking a "step back" question. This can be combined with regular question-answering applications by then doing retrieval on both the original and step-back question.

Read the paper [here](https://arxiv.org/abs/2310.06117)

See an excellent blog post on this by Cobus Greyling [here](https://cobusgreyling.medium.com/a-new-prompt-engineering-technique-has-been-introduced-called-step-back-prompting-b00e8954cacb)

In this cookbook we will replicate this technique. We modify the prompts used slightly to work better with chat models.

First, define the step back prompts & examples. This will be used later when querying the model to generate a better query based on the users original question.

In [1]:
import {
  ChatPromptTemplate,
  FewShotChatMessagePromptTemplate,
} from "npm:langchain@0.0.177/prompts";

In [2]:
const examples = [
  {
    input: "Could the members of The Police perform lawful arrests?",
    output: "what can the members of The Police do?",
  },
  {
    input: "Jan Sindel's was born in what country?",
    output: "what is Jan Sindel's personal history?",
  },
];
const examplePrompt = ChatPromptTemplate.fromMessages([
  ["human", "{input}"],
  ["ai", "{output}"],
]);
const fewShotPrompt = new FewShotChatMessagePromptTemplate({
  examplePrompt,
  examples,
  inputVariables: [], // no input variables
});

Next, format the few shot prompt as a string so we can pass it into the next step for the prompt template.

In [3]:
const formattedFewShot = await fewShotPrompt.format({});

In [4]:
const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:`,
  ],
  formattedFewShot,
  ["user", "{question}"],
]);

## Question Generator Chain

Now construct the question generator using a RunnableSequence. This will be the chain which actually generates the updated query.

In [5]:
import { ChatOpenAI } from "npm:langchain@0.0.177/chat_models/openai";
import { StringOutputParser } from "npm:langchain@0.0.177/schema/output_parser";
import { RunnableSequence } from "npm:langchain@0.0.177/schema/runnable";

In [6]:
// Deno.env.set("OPENAI_API_KEY", "");

const model = new ChatOpenAI({ model: "o4-mini", temperature: 0 });
const stringOutputParser = new StringOutputParser();

In [7]:
const questionGenerator = RunnableSequence.from([
  prompt,
  model,
  stringOutputParser,
]);

In [8]:
const question = "was chatgpt around while trump was president?";

In [18]:
await questionGenerator.invoke({ question })

"What is the timeline of ChatGPT's existence?"

## Search Retriever
Next, we'll use the SerpAPI as a search tool, and turn it into a retriever using a simple wrapper.

In [10]:
import { SerpAPI } from "npm:langchain@0.0.177/tools";

In [11]:
const search = new SerpAPI(Deno.env.get("SERPAPI_API_KEY"), {
  num: "4", // Number of results
});

In [12]:
const retriever = async (query: string) => search.call(query);

Then, pull in a predefined step-back prompt from the [LangChain Hub](https://smith.langchain.com/hub)

In [14]:
import { pull } from "npm:langchain@0.0.177/hub";

In [15]:
// const responsePromptTemplate = `You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

// {normal_context}
// {step_back_context}

// Original Question: {question}
// Answer:`;
// const responsePrompt = ChatPromptTemplate.fromTemplate(responsePromptTemplate)
const responsePrompt = await pull("langchain-ai/stepback-answer");

## Final Chain
Lastly, define our chain which will pull all the above pieces together.

In [16]:
const chain = RunnableSequence.from([
  {
    normal_context: (i: { question: string }) => retriever(i.question),
    step_back_context: questionGenerator.pipe(retriever),
    question: (i: { question: string }) => i.question,
  },
  responsePrompt,
  model,
  stringOutputParser,
]);

In [17]:
await chain.invoke({
  question,
});

"Yes, ChatGPT was around while Donald Trump was president. The exact timeline of ChatGPT's developmen"... 415 more characters